# Dependência

In [ ]:
# Install required Google Cloud packages (commented out as these are typically one-time setup commands)
# !pip install gcloud
# !gcloud auth application-default login

# Import necessary Python libraries
import pandas as pd                # Data manipulation and analysis
import numpy as np                 # Numerical computing
import time                        # Time-related functions
import os                          # Operating system interfaces
import pandas_gbq                  # Pandas integration with BigQuery
from google.cloud import bigquery  # BigQuery client library
import glob                        # File path pattern matching
import openpyxl                    # Excel file handling
import csv                         # CSV file handling
import re                          # Regular expressions

# Note: The actual imports remain exactly as in the original code

# Tratamento

In [ ]:
# Load data from Excel file (MUNIC 2023 survey, Human Rights sheet)
df = pd.read_excel("Base_MUNIC_2023(1).xlsx", sheet_name='Direitos Humanos')

# Select specific columns from the dataset
df = df[['Uf', 'Cod Uf', 'Cod Munic', 'Desc Mun', 'MDHU04', 'MDHU05', 'MDHU06', 'MDHU07']]

# Add year column with value 2023
df['ano'] = 2023

# Rename columns to more descriptive names
df = df.rename(columns={
    'Uf': 'sigla_uf',
    'Cod Uf': 'cod_uf',
    'Cod Munic': 'id_municipio',
    'Desc Mun': 'nome_municipio',
    'MDHU04': 'genero',
    'MDHU05': 'idade',
    'MDHU06': 'cor_raca',
    'MDHU07': 'grau_instrucao'
})

# Standardize race/color category (change 'Pardo' to 'Parda')
df['cor_raca'] = np.where(df['cor_raca'] == 'Pardo', 'Parda', df['cor_raca'])

# Handle missing/refused data for gender column
df['genero'] = np.where(df['genero'] == 'Recusa', 'Sem dados', df['genero'])
df['genero'] = np.where(df['genero'] == 'Não informou', 'Sem dados', df['genero'])
df['genero'] = np.where(df['genero'] == '-', 'Sem dados', df['genero'])
df['genero'] = np.where(df['genero'] == '(**) Sem gestor', 'Sem dados', df['genero'])
df['genero'] = np.where(df['genero'] == 'Não soube informar', 'Sem dados', df['genero'])

# Handle missing/refused data for race/color column
df['cor_raca'] = np.where(df['cor_raca'] == 'Recusa', 'Sem dados', df['cor_raca'])
df['cor_raca'] = np.where(df['cor_raca'] == 'Não informou', 'Sem dados', df['cor_raca'])
df['cor_raca'] = np.where(df['cor_raca'] == '-', 'Sem dados', df['cor_raca'])
df['cor_raca'] = np.where(df['cor_raca'] == '(**) Sem gestor', 'Sem dados', df['cor_raca'])
df['cor_raca'] = np.where(df['cor_raca'] == 'Não soube informar', 'Sem dados', df['cor_raca'])

# Handle missing/refused data for education level column
df['grau_instrucao'] = np.where(df['grau_instrucao'] == 'Recusa', 'Sem dados', df['grau_instrucao'])
df['grau_instrucao'] = np.where(df['grau_instrucao'] == 'Não informou', 'Sem dados', df['grau_instrucao'])
df['grau_instrucao'] = np.where(df['grau_instrucao'] == '-', 'Sem dados', df['grau_instrucao'])
df['grau_instrucao'] = np.where(df['grau_instrucao'] == '(**) Sem gestor', 'Sem dados', df['grau_instrucao'])
df['grau_instrucao'] = np.where(df['grau_instrucao'] == 'Não soube informar', 'Sem dados', df['grau_instrucao'])

# Handle missing/refused data for age column (convert to NaN)
df['idade'] = np.where(df['idade'] == 'Recusa', np.nan, df['idade'])
df['idade'] = np.where(df['idade'] == 'Não informou', np.nan, df['idade'])
df['idade'] = np.where(df['idade'] == '-', np.nan, df['idade'])
df['idade'] = np.where(df['idade'] == '(**) Sem gestor', np.nan, df['idade'])
df['idade'] = np.where(df['idade'] == 'Não soube informar', np.nan, df['idade'])

# Convert age column to numeric type
df['idade'] = pd.to_numeric(df['idade'])

# Define age group bins and labels
limites = [18, 30, 50, 65, 100]
categorias = ['Entre 18-29', 'Entre 30-49', 'Entre 50-64', 'Acima de 65']

# Create age group column based on the bins
df['faixa_etaria'] = pd.cut(df['idade'], bins=limites, labels=categorias, right=False)

# Reorder columns in the dataframe
df = df[['ano', 'sigla_uf', 'nome_municipio', 'id_municipio', 'genero', 'cor_raca', 'grau_instrucao', 'faixa_etaria']]

# Create dictionary to standardize education levels
dict_esco = {
    'Ensino fundamental incompleto': 'Até Ensino Fundamental',
    'Ensino fundamental completo': 'Até Ensino Fundamental',
    'Ensino fundamental (1º Grau) completo': 'Até Ensino Fundamental',
    'Ensino fundamental (1º Grau) incompleto': 'Até Ensino Fundamental',
    'Ensino médio (2º Grau) incompleto': 'Até Ensino Médio',
    'Ensino médio (2º Grau) completo': 'Até Ensino Médio',
    'Ensino superior incompleto': 'Até Ensino Superior Completo',
    'Ensino superior completo': 'Até Ensino Superior Completo',
    'Especialização': 'Até Pós Graduação ou Mestrado',
    'Mestrado': 'Até Pós Graduação ou Mestrado',
    'Doutorado': 'Até Doutorado'
}

# Apply education level standardization
df = df.replace({'grau_instrucao': dict_esco})

# Display unique values in education column for verification
df['grau_instrucao'].unique()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   ano             5570 non-null   int64   
 1   sigla_uf        5570 non-null   object  
 2   nome_municipio  5570 non-null   object  
 3   id_municipio    5570 non-null   int64   
 4   genero          5570 non-null   object  
 5   cor_raca        5570 non-null   object  
 6   grau_instrucao  5570 non-null   object  
 7   faixa_etaria    1893 non-null   category
dtypes: category(1), int64(2), object(5)
memory usage: 310.4+ KB


# Upload

In [ ]:
# Define the BigQuery table schema with field descriptions (in Portuguese)
schema = [
    bigquery.SchemaField('ano', 'INTEGER', description='Ano de referência da observação'),
    bigquery.SchemaField('sigla_uf', 'STRING', description='Sigla da Unidade da Federação referente municipio'),
    bigquery.SchemaField('nome_municipio', 'STRING', description='Nome do município'),
    bigquery.SchemaField('id_municipio', 'INTEGER', description='Código do IBGE do município'),
    bigquery.SchemaField('genero', 'STRING', description='Gênero autodeclarado ou não'),
    bigquery.SchemaField('cor_raca', 'STRING', description='Raça/cor da pessoa observada'),
    bigquery.SchemaField('grau_instrucao', 'STRING',
                       description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação'),
    bigquery.SchemaField('faixa_etaria', 'STRING', description='faixa etária da observação')
]

# Initialize BigQuery client connection to the specified project
client = bigquery.Client(project='repositoriodedadosgpsp')

# Create reference to the target dataset 'cargos_lideranca'
dataset_ref = client.dataset('cargos_lideranca')

# Create reference to the target table with standardized naming convention:
# Format: FONTE_algo_intuitivo_dado (MUNIC_perfil_gestor_direitos_humanos_v3)
table_ref = dataset_ref.table('MUNIC_perfil_gestor_direitos_humanos_v3')

# Configure the load job with our schema definition
job_config = bigquery.LoadJobConfig(schema=schema)

# Execute the load job to transfer DataFrame to BigQuery
job = client.load_table_from_dataframe(
    dataframe=df,
    destination=table_ref,
    job_config=job_config
)

# Wait for the job to complete
job.result()